In [1]:
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

from glob import glob
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
print(os.getcwd())

D:\Github\Stomach-Status-Classification\Utils


In [3]:
os.chdir("..")
print(os.getcwd())

D:\Github\Stomach-Status-Classification


In [4]:
exper_dir = os.getcwd() + "\\Experiment"
print(os.listdir(exper_dir))

['Approach', 'Approach2-8', 'DLApproach', 'README.md', 'Results bag of visual words-20221019T081806Z-001', 'Training']


In [5]:
approach_dir = exper_dir + "\\Approach"
print(os.listdir(approach_dir))

['all_model_analysis', 'DecisionTree', 'ExtraTreeClassifier', 'HARDMSEG', 'LogisticRegression', 'RandomForest', 'SVM']


In [6]:
method = [x for x in os.listdir(approach_dir) if x not in ["HARDMSEG", "all_model_analysis"]]
print(method)

['DecisionTree', 'ExtraTreeClassifier', 'LogisticRegression', 'RandomForest', 'SVM']


In [7]:
method_dirs = [approach_dir + f"\\{x}" for x in method]
method_dirs

['D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\DecisionTree',
 'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\ExtraTreeClassifier',
 'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\LogisticRegression',
 'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\RandomForest',
 'D:\\Github\\Stomach-Status-Classification\\Experiment\\Approach\\SVM']

In [8]:
print(os.listdir(method_dirs[0]))

['dt_km_akaze_10cv_128.ipynb', 'dt_km_akaze_10cv_256.ipynb', 'dt_km_akaze_10cv_64.ipynb', 'dt_km_brisk_10cv_128.ipynb', 'dt_km_brisk_10cv_256.ipynb', 'dt_km_brisk_10cv_64.ipynb', 'dt_km_kaze_10cv_128.ipynb', 'dt_km_kaze_10cv_256.ipynb', 'dt_km_kaze_10cv_64.ipynb', 'dt_km_orb_10cv_128.ipynb', 'dt_km_orb_10cv_256.ipynb', 'dt_km_orb_10cv_64.ipynb', 'dt_km_sift_10cv_128.ipynb', 'dt_km_sift_10cv_256.ipynb', 'dt_km_sift_10cv_64.ipynb', 'dt_km_surf0_10cv_1024.ipynb', 'dt_km_surf0_10cv_128.ipynb', 'dt_km_surf0_10cv_256.ipynb', 'dt_km_surf0_10cv_512.ipynb', 'dt_km_surf0_10cv_64.ipynb', 'dt_km_surf100_10cv_128.ipynb', 'dt_km_surf100_10cv_256.ipynb', 'dt_km_surf100_10cv_64.ipynb', 'dt_km_surf50_10cv_128.ipynb', 'dt_km_surf50_10cv_256.ipynb', 'dt_km_surf50_10cv_512.ipynb', 'dt_km_surf50_10cv_64.ipynb', 'dt_km_surf_10cv_128.ipynb', 'dt_km_surf_10cv_64.ipynb', 'rdf_km1024_surf0_result_fold10_0.9.pkl', 'rdf_km128_akaze_result_fold10_0.9.pkl', 'rdf_km128_brisk_result_fold10_0.9.pkl', 'rdf_km128_kaze_r

In [9]:
pkls = []

for method in method_dirs:
    base_pkl = [x for x in glob(method + "\\*.pkl") if "512" not in x and "1024" not in x and "32" not in x]
    pkls += base_pkl

print(len(pkls))

135


In [10]:
ipys = []

for method in method_dirs:
    base_ipy = [x for x in glob(method + "\\*.ipynb") if "512" not in x and "1024" not in x and "32" not in x]
    ipys += base_ipy

print(len(ipys))

134


In [11]:
pkls_dict = {
    "sift" : [x for x in pkls if "sift" in x],
    "surf" : [x for x in pkls if "surf" in x],
    "brisk" : [x for x in pkls if "brisk" in x],
    "orb" : [x for x in pkls if "orb" in x],
    "kaze" : [x for x in pkls if "kaze" in x and "akaze" not in x],
    "akaze" : [x for x in pkls if "akaze" in x]
}

for key in pkls_dict:
    print(key, len(pkls_dict[key]))

sift 15
surf 60
brisk 15
orb 15
kaze 15
akaze 15


In [12]:
ipy_dict = {
    "sift" : [x for x in ipys if "sift" in x],
    "surf" : [x for x in ipys if "surf" in x],
    "brisk" : [x for x in ipys if "brisk" in x],
    "orb" : [x for x in ipys if "orb" in x],
    "kaze" : [x for x in ipys if "kaze" in x and "akaze" not in x],
    "akaze" : [x for x in ipys if "akaze" in x]
}

for key in ipy_dict:
    print(key, len(ipy_dict[key]))

sift 15
surf 59
brisk 15
orb 15
kaze 15
akaze 15


In [13]:
sift_dict = {
    "method" : [],
    "cluster" : [],
    "fold" : [],
    "accuracy" : [],
    "precision" : [],
    "sensitivity" : [],
    "f1-score" : [],
    "specificity" : [],
    "auc" : []
}

method_mapping = {
    'DecisionTree' : "DT",
    'ExtraTreeClassifier' : "ETC",
    'LogisticRegression' : "LR",
    'RandomForest' : "RF",
    'SVM' : "SVM"
}

for pkl_path in pkls_dict["kaze"]:
    split = pkl_path.split("\\")
    method = split[-2]
    filename = split[-1]
#     print(method, filename)
    
    filename_split = filename.split("_")
#     print(filename_split)
    
    feature_descriptor = filename_split[2]
    
    cluster_count = filename_split[1][2:]
    
    correlated_ipy = [x for x in ipy_dict["kaze"] if feature_descriptor in x and cluster_count in x and method in x][0]
    
    ipy_json = json.load(open(correlated_ipy))
    
    ipy_json_cells = ipy_json["cells"]
    
    for cell in ipy_json_cells:
        if "id_max" in cell["source"][0]:
            print(correlated_ipy)
            train_max = cell["outputs"][0]["text"][0][:-1]
            break
            
    with open(pkl_path, "rb") as input_file:
        base_pkl_dict = pickle.load(input_file)
    
    base_pkl_train_max = base_pkl_dict[train_max]["train_fold"]
    
    for fold in base_pkl_train_max:
        fold_index = fold[-1]
        
        fold_data = base_pkl_train_max[fold]
        roc_auc_score = fold_data["ROC_AUC_SCORE"]["Macro"]
        cls_report = fold_data["Classification Report"]
        
        sift_dict["method"].append(method_mapping[method])
        sift_dict["cluster"].append(cluster_count)
        sift_dict["fold"].append(fold_index)
        sift_dict["accuracy"].append(cls_report["accuracy"])
        sift_dict["precision"].append(cls_report["weighted avg"]["precision"])
        sift_dict["sensitivity"].append(cls_report["weighted avg"]["recall"])
        sift_dict["f1-score"].append(cls_report["weighted avg"]["f1-score"])
        sift_dict["specificity"].append(fold_data["Confusion Matrix"]["specificity"])
        sift_dict["auc"].append(roc_auc_score)

D:\Github\Stomach-Status-Classification\Experiment\Approach\DecisionTree\dt_km_kaze_10cv_128.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\DecisionTree\dt_km_kaze_10cv_256.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\DecisionTree\dt_km_kaze_10cv_64.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\ExtraTreeClassifier\etc_km_kaze_10cv_128.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\ExtraTreeClassifier\etc_km_kaze_10cv_256.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\ExtraTreeClassifier\etc_km_kaze_10cv_64.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\LogisticRegression\lr_km_kaze_10cv_128.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\LogisticRegression\lr_km_kaze_10cv_256.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\LogisticRegression\lr_km_kaze_10cv_64.ipynb
D:\Github\Stomach-Status-Classification\Experiment\Approach\RandomFore

IndexError: list index out of range

In [ ]:
sift_dict_df = pd.DataFrame(sift_dict)
sift_dict_df.head()

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="accuracy", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="precision", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="sensitivity", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="f1-score", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="specificity", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)

In [ ]:
sns.set_theme(style="ticks")
sns.boxplot(x="method", y="auc", hue = "cluster", data=sift_dict_df,
            whis=[0, 100], width=0.6, palette="vlag")
sns.despine(trim=True, left=True)